<a href="https://colab.research.google.com/github/SlySlayer32/Kidsmessyroom/blob/main/Kid_Room_Cleanup_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =====================================
# STEP 1: Install Dependencies
# =====================================
!pip install ultralytics
!pip install segment-anything-py
!pip install roboflow
!pip install opencv-python
!pip install pillow
!pip install pandas
!pip install matplotlib

In [ ]:
# =====================================
# STEP 2: Import Libraries
# =====================================
import cv2
import numpy as np
from ultralytics import YOLO
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
import json

print("✅ All libraries imported successfully!")

In [ ]:
# =====================================
# STEP 3: Load YOLOv8 Model
# =====================================
# Load the pre-trained YOLOv8 model (you can use yolov8n, yolov8s, yolov8m, yolov8l, yolov8x)
model = YOLO('yolov8n.pt')  # 'n' = nano (fastest, smallest)

print("✅ YOLOv8 model loaded successfully!")

In [ ]:
# =====================================
# STEP 4: Object Detection & Cataloging Function
# =====================================
def detect_and_catalog_objects(image_path, confidence_threshold=0.25):
    """
    Detect all objects in an image and create a catalog with counts.
    Perfect for messy rooms with many scattered items.
    """
    # Read image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Run detection
    results = model(image_path, conf=confidence_threshold)

    # Extract detection data
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = int(box.cls[0].cpu().numpy())
            class_name = model.names[cls]

            detections.append({
                'class': class_name,
                'confidence': float(conf),
                'bbox': [int(x1), int(y1), int(x2), int(y2)]
            })

    # Create catalog with counts
    object_counts = Counter([d['class'] for d in detections])

    # Create DataFrame for easy viewing
    catalog_df = pd.DataFrame([
        {'Object': obj, 'Count': count}
        for obj, count in object_counts.most_common()
    ])

    return detections, catalog_df, image_rgb

print("✅ Detection function ready!")

In [ ]:
# =====================================
# STEP 5: Visualization Function
# =====================================
def visualize_detections(image_rgb, detections, catalog_df):
    """
    Visualize detected objects with bounding boxes and show catalog.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

    # Draw bounding boxes
    img_with_boxes = image_rgb.copy()
    for det in detections:
        x1, y1, x2, y2 = det['bbox']
        cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), (0, 255, 0), 3)
        label = f"{det['class']} {det['confidence']:.2f}"
        cv2.putText(img_with_boxes, label, (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    # Show image
    ax1.imshow(img_with_boxes)
    ax1.set_title(f'Detected Objects (Total: {len(detections)})', fontsize=16)
    ax1.axis('off')

    # Show catalog as table
    ax2.axis('tight')
    ax2.axis('off')
    table = ax2.table(cellText=catalog_df.values,
                      colLabels=catalog_df.columns,
                      cellLoc='center',
                      loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2)
    ax2.set_title('Object Catalog', fontsize=16)

    plt.tight_layout()
    plt.show()

    print(f"\n✅ Found {len(detections)} objects!")
    print(f"\n📊 Object Summary:")
    print(catalog_df.to_string(index=False))

print("✅ Visualization function ready!")

In [ ]:
# =====================================
# STEP 6: Example Usage
# =====================================
# Upload your image and run detection

# Example: Upload an image of a messy room
from google.colab import files
uploaded = files.upload()

# Get the uploaded image path
image_path = list(uploaded.keys())[0]

print(f"✅ Image uploaded: {image_path}")
print("\n🔍 Running object detection...\n")

# Run detection and cataloging
detections, catalog, image_rgb = detect_and_catalog_objects(image_path, confidence_threshold=0.25)

# Visualize results
visualize_detections(image_rgb, detections, catalog)

# Save catalog to CSV
catalog.to_csv('object_catalog.csv', index=False)
print("\n💾 Catalog saved to 'object_catalog.csv'")

In [ ]:
# =====================================
# NEXT STEPS: Build Your Complete Pipeline
# =====================================

print("""
🚀 CONGRATULATIONS! You've set up the foundation!

📌 NEXT STEPS FOR YOUR KID'S ROOM CLEANUP APP:

1️⃣ FINE-TUNE FOR TOY DETECTION:
   - Collect 100-200 photos of messy kids' rooms
   - Label them using Roboflow: https://roboflow.com
   - Train a custom YOLOv8 model on toy categories:
     * Dolls, action figures, books, blocks, cars, etc.
   - Use: model.train(data='your_dataset.yaml', epochs=100)

2️⃣ ADD CARTOON/GAME STYLE TRANSFER:
   - Install: !pip install diffusers transformers
   - Use Stable Diffusion ControlNet or American Cartoon LoRA
   - Models to try:
     * Kontext-Style/American_Cartoon_lora
     * sd-dreambooth-library/smiling-friendly-kids-cartoon

3️⃣ IMPLEMENT OBJECT REMOVAL/INPAINTING:
   - Install: !pip install lama-cleaner
   - Use LaMa or Stable Diffusion Inpainting
   - Mask detected objects and inpaint background

4️⃣ CREATE INTERACTIVE UI:
   - Use Gradio or Streamlit for web interface
   - Add drag-and-drop for detected objects
   - Create game mechanics (points for cleanup)

5️⃣ BUILD MOBILE APP (Flutter/React Native):
   - Use camera to take photos
   - Send to your API for detection
   - Display cartoon version with interactive objects
   - Add physics-based dragging (like Toca Boca)

📚 RESOURCES:
- YOLOv8 Docs: https://docs.ultralytics.com
- Roboflow: https://roboflow.com
- Hugging Face Spaces: https://huggingface.co/spaces
- SAM (Segment Anything): https://github.com/facebookresearch/segment-anything
- Label Studio: https://labelstud.io

👨‍💻 TIPS FOR YOUR 8-YEAR-OLD APP:
- Use bright colors and big buttons
- Add sound effects for each action
- Reward system (stars/badges)
- Before/after comparison feature
- Parent dashboard for tracking progress

✅ This notebook gives you the FOUNDATION.
🛠️ Now customize it for your specific use case!
""")

print("\n🎉 Ready to detect objects in messy rooms! Upload an image above to test.")